# Classification of the MNIST-dataset

Classification of the images in the hand-drawn MNIST-dataset has been a benchmark for models for years. The dataset contains about 60 000 training images and 10 000 test images. This might seem like a lot but considering the fact that the average neural network used for classification on this dataset is comprised of about 200 000 parameters, it is rather interesting to get the accuracy we do. In this notebook, I will be exploring the accuracy of predictions of a basic neural network and one with convolutions.

This notebook is the first Convolutional Neural Network I have written and as such, the daunting task of tuning the layers proved quite a struggle. The following, however, was the process I went through:

#### Convolutions:

Almost all CNNs start with a convolution layer. We aim to extract special patterns (or features) from the images to better emulate the way humans classify images. These convolutions come with quite a lot of tuning, however. We start with channels. The MNIST-dataset is grayscale and thus each image has one channel. The out-channel is the amount of filters to be applied through the layer and can be conceptualized as the amount of different patterns we would like to learn in this layer. Increasing this hyperparameter increases the complexity of the model and can lead to overfitting. It is common to choose 16 or 32 but without some experimenting, the best choice is unclear. Next, we have to choose the size of the filter, how much it moves and padding of the image. Using reference (2), we can determine the hyperparameters such that all pixels on the image are used to the same magnitude. The size will most likely be 3x3 with our image size (28x28), then with stride of 1, we need padding of 1 on each side.

#### Batch-Normalization and Dropout
A quick search on google scholar on the importance of BN and Dropout and where to put them highlights the ongoing debate on this matter. Dropout is the process of choosing neurons at random and ignoring their output. This process aims to reduce overfitting by encouraging the network to rely on multiple neurons for specific patterns. Batch-Normalization aims to increase the convergence rate on a minimal loss by decreasing the internal covariance shift. The original paper written by Loffe and Szegedy suggests putting Batch normalization before activation functions. This, however doesn't make sense as it leads defies the point of BN, as supported by reference 4. Also, reference 5 (and many tests) suggest that using dropout regularization with BN leads to worse results compared to using only BN and as such, BN will be used without DR and the BN-layers will be put after the activation layers.


In [1]:
import MNIST_classifier as mn
from torch import nn
import torch
training_dataloader, testing_dataloader = mn.init_MNIST(True)


0.13066047430038452
0.30810782313346863


In [2]:
model = mn.CNN().cuda()

In [3]:
model.train_from_dataset(4, training_dataloader, testing_dataloader)

Epoch 1

Test Error: 
 Accuracy: 98.1%, Avg loss: 0.059227 

Epoch 2

Test Error: 
 Accuracy: 98.7%, Avg loss: 0.047521 

Epoch 3

Test Error: 
 Accuracy: 98.8%, Avg loss: 0.096994 

Epoch 4

Test Error: 
 Accuracy: 98.6%, Avg loss: 0.046884 



In [5]:
#torch.save(model, "CNN.pt")

## References

<ol>
<li>Zhang, A., Lipton, Z., li, M. and Smola, A. (2023). Dive Into Deep Learning. [online] Cambridge University Press. Available at: https://d2l.ai.</li>
<li>asiltureli.github.io. (n.d.). ConvNet Size Calculator. [online] Available at: https://asiltureli.github.io/Convolution-Layer-Calculator/ [Accessed 4 May 2024].</li>
<li>Loffe, S. and Szegedy, C. (2015). Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift. [online] Available at: https://arxiv.org/abs/1502.03167 [Accessed 4 May 2024].</li>
<li>Rosebrock, A. (2021). Convolutional Neural Networks (CNNs) and Layer Types. [online] PyImageSearch. Available at: https://pyimagesearch.com/2021/05/14/convolutional-neural-networks-cnns-and-layer-types/.</li>
<li>Li, X., Chen, S., Hu, X. and Yang, J. (2018). Understanding the Disharmony between Dropout and Batch Normalization by Variance Shift. [online] Available at: https://arxiv.org/pdf/1801.05134 [Accessed 4 May 2024].</li>
</ol>